![logo](check_point_logo.png)

# Tutorial on manipulating MSI L1C product


In [ ]:
from pathlib import Path

import cartopy.crs as ccrs  # For static plotting
import cartopy.feature as cf
import geopandas  # For interactive plotting
import numpy as np
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import xarray as xr

from eopf.store.safe2 import XarrayStore
from eopf.store.safe2 import EOSafeStore2
from eopf.common.constants import OpeningMode
from eopf.product.eo_variable import EOVariable

Change directory to EOPF CPM source code ....

In [ ]:
import os
path = '/mount/internal/work-st/projects/cs-412/2078-dpr/workspace/vlevasseur/Projects/eopf-cpm'
old_path = os.getcwd()
os.chdir(path)


product can only be accessed locally

In [ ]:
SAMPLE_PATH = Path("/mount/internal/work-st/projects/cs-412/2078-dpr/Samples/Products/Zarr_Beta")
files = [ f for f in SAMPLE_PATH.glob("S02MSIL1C*.zarr")]
product = files[0]
product

### Open the product


In [ ]:
xs = XarrayStore(str(product))
xs.open()
eop=xs.load()
eop


### Overview of the product content

In [ ]:
eop["conditions/detfoo/r60m"]["b01"].plot()

In [ ]:
eop["measurements/r60m"]["b01"].plot()

In [ ]:
eop["measurements/r60m"]["b01"]._data.encoding["add_offset"]

In [ ]:
min,max,mean=eop["measurements/r60m"]["b01"].data.max(), eop["measurements/r60m"]["b01"].data.min(), eop["measurements/r60m"]["b01"].data.mean()
min.compute(),max.compute(),mean.compute()

In [ ]:
r = eop["measurements/r10m"]["b04"]
g = eop["measurements/r10m"]["b03"]
b = eop["measurements/r10m"]["b02"]

xr.Dataset(dict(r=r._data, g=g._data, b=b._data))

In [ ]:
b02 = eop["measurements/r10m"]["b02"]
b02

In [ ]:
b02.dtype

In [ ]:
b02._data.encoding

## Plot a RGB image

In [ ]:
rgb_band_paths = (
            f"measurements/r10m/b04",
            f"measurements/r10m/b03",
            f"measurements/r10m/b02",
        )

concat = xr.concat(
        [eop[str(p)]._data for p in rgb_band_paths],  # type: ignore
        dim="band",
    )

ax = concat.plot.imshow()
ax.axes.set_aspect("equal")
plt.gca().invert_yaxis()

### Explore product geolocation

The following snippet shows an interactive map with the tile's footprint


#### Interactive map

In [ ]:
gdf = geopandas.GeoDataFrame.from_features([eop.attrs["stac_discovery"]])

Note: CRS is missing from the metadata, it must be set manually


In [ ]:
gdf = gdf.set_crs(4326)

In [ ]:
gdf.explore()

In [ ]:
gdf.crs

#### Non-interactive map

The following snippet shows the location of the tile on a global map.


In [ ]:
def main():
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())

    # make the map global rather than have it zoom in to
    # the extents of any plotted data
    ax.set_global()

    # ax.stock_img()
    ax.coastlines()

    ax.plot(gdf.centroid[0].x, gdf.centroid[0].y, "ro", transform=ccrs.PlateCarree())

    plt.show()


if __name__ == "__main__":
    main()

#### Plot georeferenced data

In [ ]:
# Define constant for plotting
L1C_PROJECTION = ccrs.epsg(32633)
DESIRED_PROJECTION = ccrs.PlateCarree()
FIGSIZE: tuple[int, int] = (12, 8)
RESOLUTION_CARTOPY: str = '110m'
GEOGRAPHICAL_LIMITS: tuple[int, int, int, int] = (-20, 30, 10, 30)
GEOGRAPHICAL_LIMITS: tuple[int, int, int, int] = (0, 10, 42, 46)

# Speed up plot by sampling data every SKIP_EVERY pixels
SKIP_EVERY: int = 50

# Define plotting arguments for Polygon around the area of interest
POLYGON_THICKNESS: int = 1
POLYGON_COLOR: str = 'r'

# Get the geometry from the product and check that it correspond to the domain represented
geometry_from_product = np.squeeze(eop.attrs["stac_discovery"]["geometry"]["coordinates"])
geometry_from_product

In [ ]:
_, ax = plt.subplots(subplot_kw={"projection": DESIRED_PROJECTION},
                    figsize=FIGSIZE)

# Plot cartopy geographic information
ax.coastlines(resolution=RESOLUTION_CARTOPY)
ax.add_feature(cf.BORDERS)
ax.add_feature(cf.OCEAN)
ax.add_feature(cf.LAND)
gl = ax.gridlines(draw_labels=True, 
                  crs=DESIRED_PROJECTION)


b02 = eop["measurements/r10m"]["b02"]
plt.contourf(b02[::SKIP_EVERY, ::SKIP_EVERY], transform=L1C_PROJECTION)
poly = mpatches.Polygon(geometry_from_product, 
                        closed=True, 
                        ec=POLYGON_COLOR, 
                        fill=False, 
                        lw=POLYGON_THICKNESS, 
                        transform=DESIRED_PROJECTION)
ax.add_patch(poly)
ax.set_extent(GEOGRAPHICAL_LIMITS, crs=DESIRED_PROJECTION)
cbar = plt.colorbar(orientation="horizontal")
cbar.set_label('b02_10m')
plt.tight_layout()

## Compute radiances

From: https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/product-types/level-1c

$radiance = reflectance * \cos(radians(SunZenithAngle)) * solarIrradiance * U / pi$

In [ ]:
U: float = eop.attrs["other_metadata"]["reflectance_correction_factor_from_the_Sun-Earth_distance_variation_computed_using_the_acquisition_date"]
U

In [ ]:
# Be carefull, Sun Zenith Angle is expressed on the angles grid (5km), it needs to be reprojected on the 10m grid for computing radiances
# cosinus is applied now because we can not interpolate angles using a linear interpolation (discontinuity at 0°)
# On the other hand, cosines can be interpolated
# cos_zsa_5km: EOVariable = np.cos(np.deg2rad(xdt[].conditions.geometry.sza))
# cos_zsa_5km
cos_sza_5km: EOVariable = np.cos(np.deg2rad(eop["conditions/sun_ang/zen"]))
cos_sza_5km

In [ ]:
# We will convert reflectances from band BAND to radiances
BAND: int = 2

# Band - 1 because Python list index starts at 0
solarIrradiance: float = np.float64(eop.attrs["stac_discovery"]["properties"]["eo:bands"][BAND-1]["solar_illumination"])

In [ ]:
reflectance_b02_10m: EOVariable = eop["measurements/r10m/b02"]
reflectance_b02_10m

In [ ]:
# Interpolate sza on the angles grid to the 10m grid
cos_sza_10m: xr.DataArray = cos_sza_5km._data.interp_like(reflectance_b02_10m._data)
cos_sza_10m.compute()

In [ ]:
# For simplicity, radiance computation assume that reflectances equal numerical counts
radiance = reflectance_b02_10m._data * cos_sza_10m * solarIrradiance * U / np.pi
radiance

Visualize computational graph
(Needs to have graphviz package)

In [ ]:
radiance.data.visualize()

In [ ]:
radiance.plot()